In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotZoo
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random
using Test
include("quadratic_cost.jl")
include("walker.jl")
include("sparseblocks.jl")
include("utils.jl")
const isautograder = @isdefined autograder;

  Activating environment at `~/devel/optimal_control/hw4_solutions/Project.toml`
┌ Info: Precompiling ForwardDiff [f6369f11-7733-5829-9624-2563aa707210]
└ @ Base loading.jl:1342
┌ Info: Precompiling RobotZoo [74be38bb-dcc2-4b9e-baf3-d6373cd95f10]
└ @ Base loading.jl:1342
┌ Info: Precompiling TrajOptPlots [7770976a-8dee-4930-bf39-a1782fd21ce6]
└ @ Base loading.jl:1342


WebIO._IJuliaInit()

![](../walker.gif)

# Q2: Hybrid Trajectory Optimization  (40 pts)
In this problem you'll use a direct method to optimize a walking trajectory for a simple biped model, using the hybrid dynamics formulation. You'll pre-specify a gait sequence and solve the problem using Ipopt, a high-quality open-source interior point nonlinear programming solver (actually developed here at CMU!). Your final solution should look like the video above.

## The Dynamics
Our system is modeled as three point masses: one for the body and one for each foot. The state is defined as the x and y positions and velocities of these masses, for a total of 6 degrees of freedom and 12 states. The legs are connected to the body with prismatic joints. The system has three control inputs: a force along each leg, and the torque between the legs.
Reference the code block below for a quick overview of the API we've implemented for you. You're encouraged to look at the code in [src/walker.jl](https://github.com/Optimal-Control-16-745/hw4_solutions/blob/master/src/walker.jlhttps://github.com/Optimal-Control-16-745/hw4_solutions/blob/master/src/walker.jl).

The state and control vectors are ordered as follows:

$$ x = \begin{bmatrix} 
    p_x^{(b)} \\ p_y^{(b)} \\ p_x^{(1)} \\ p_y^{(1)} \\ p_x^{(2)} \\ p_y^{(2)} \\
    v_x^{(b)} \\ v_y^{(b)} \\ v_x^{(1)} \\ v_y^{(1)} \\ v_x^{(2)} \\ v_y^{(2)} \\
\end{bmatrix} \quad
u = \begin{bmatrix} F^{(1)} \\ F^{(2)} \\ \tau \end{bmatrix}
$$
where e.g. $p_x^{(b)}$ is the $x$ position of the body, $v_y^{(i)}$ is the $y$ velocity of foot $i$, $F^{(i)}$ is the force along leg $i$, and $\tau$ is the torque between the legs.

In [2]:
model = SimpleWalker()
x,u = rand(model)  # generate some random states and controls
dt = 0.1

# evaluate the discrete dynamics using RK4
stance1_dynamics_rk4(model, x, u, dt)
stance2_dynamics_rk4(model, x, u, dt)

# jump maps
jump1_map(x)
jump2_map(x)

# evaluate the discrete dynamics Jacobians
stance1_jacobian(model, x, u, dt)
stance2_jacobian(model, x, u, dt)

# jump map Jacobian
jump1_jacobian()
jump2_jacobian();

# visualizer
x = zeros(12)
x[2] = 1
if !isautograder
    vis = Visualizer()
    set_mesh!(vis, model)
    visualize!(vis, model, SVector{12}(x))
    render(vis)
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /Users/kevintracy/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("Rfoot" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x52, 0x66, 0x6f, 0x6f, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x52, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x52, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "torso" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x74, 0x6f, 0x72, 0x73, 0x6f], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("body" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x62, 0x6f, 0x64, 0x79], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "Laxle" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x4c, 0x61, 0x78, 0x6c, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x4c, 0x61, 0x78, 0x6c, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "Lleg" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x74, 0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x4c, 0x6c, 0x65, 0x67], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x4c, 0x6c, 0x65, 0x67, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x4c, 0x6c, 0x65, 0x67, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Rleg" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x74, 0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x52, 0x6c, 0x65, 0x67], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x52, 0x6c, 0x65, 0x67, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x52, 0x6c, 0x65, 0x67, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Raxle" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x52, 0x61, 0x78, 0x6c, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x52, 0x61, 0x78, 0x6c, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Lfoot" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x4c, 0x66, 0x6f, 0x6f, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.Scen

## The Problem Formulation
The trajectory optimization problem we're solving has the following form:

$$
\begin{aligned}
&\text{minimize} && \frac{1}{2} (x_N - \bar{x}_N)^T Q_N (x_N - \bar{x}) + 
\frac{1}{2}\sum_{k=1}^{N-1} (x_k - \bar{x}_k)^T Q (x_k - \bar{x}_k) + (u_k - \bar{u}_k)^T R_k (u_k - \bar{u}_k) \\
&\text{subject to} && x_1 = x_\text{init} \\
                  &&& x_N = x_\text{goal} \\
                  &&& f_1(x_k,u_k) = x_{k+1}, && \forall k \in \mathcal{M}_1 \setminus \mathcal{J}_1 \\
                  &&& f_2(x_k,u_k) = x_{k+1}, && \forall k \in \mathcal{M}_2 \setminus \mathcal{J}_2 \\
                  &&& g_2(f_1(x_k,u_k)) = x_{k+1}, && \forall k \in \mathcal{J}_1 \\
                  &&& g_1(f_2(x_k,u_k)) = x_{k+1}, && \forall k \in \mathcal{J}_2 \\
                  &&& y^{(1)}_k = 0, && \forall k \in \mathcal{M}_1 \\
                  &&& y^{(2)}_k = 0, && \forall k \in \mathcal{M}_2 \\
                  &&& 0.5 < ||r^{(b)}_k - r^{(i)}_k|| < 1.5, && \forall k \in \mathcal{M}_i, \; i \in \{1,2\} \\
\end{aligned} 
$$
where $\bar{x}$ and $\bar{u}$ are reference states and controls. The first 2 constraints are the initial and terminal constraints, and the last constraint is a bound on the length of the prismatic leg joints.

The other constraints encode the hybrid dynamics. For this problem we have 2 different "modes," each corresponding to when one foot is on the ground (we don't consider the cases when both or neither feet are on the ground). Every knot point is assigned either to $\mathcal{M}_1$ or $\mathcal{M}_2$, but not both. To simplify our problem and obtain a nice, even walking gait, we'll assign $M$ adjacent time steps to the one mode, and then alternate. For a trajectory of 45 time steps, we'll have:
$$
\mathcal{M}_1 = \{1\text{:}5,11\text{:}15,21\text{:}25,31\text{:}35,41\text{:}45\} \\
\mathcal{M}_2 = \{6\text{:}10,16\text{:}20,26\text{:}30,36\text{:}40\}
$$

The jump map sets $\mathcal{J}_1$ and $\mathcal{J}_2$ are the indices where the mode of the next time step is different than the current, i.e. $\mathcal{J}_i \equiv \{k+1 \notin \mathcal{M}_i \; | \; k \in \mathcal{M}_i\}$. 

Lastly, constraints 7 and 8 are "stance" constraints that require that the height of the foot is zero for the corresponding mode.

## Part (a): Setting up the NLP (3 pts)
As a first step, we'll set up the variables we'll need to evaluate our constraints (we've already implemented the cost functions for you). Your constraints should be ordered as follows:

$$ \begin{bmatrix}
c_\text{init} \\
c_\text{goal} \\
c_\text{dynamics} \\
c_\text{stance} \\
c_\text{length} \\
\end{bmatrix}$$
which are of length $n$, $n$, $Nn + (N-1)m$, $N$, and $2N$, respectively. The dynamics, stance, and length constraints should be ordered by time step.

In [3]:
# TASK: Complete the constructor for the HybridNLP type (3 pts)

"""
    HybridNLP{n,m,L,Q}

Represents a (N)on(L)inear (P)rogram of a trajectory optimization problem,
with a dynamics model of type `L`, a quadratic cost function, horizon `T`, 
and initial and final state `x0`, `xf`.

The kth state and control can be extracted from the concatenated state vector `Z` using
`Z[nlp.xinds[k]]`, and `Z[nlp.uinds[k]]`.

# Constructor
    HybridNLP(model, obj, tf, N, M, x0, xf, [integration])

# Basic Methods
    Base.size(nlp)    # returns (n,m,T)
    num_ineq(nlp)     # number of inequality constraints
    num_eq(nlp)       # number of equality constraints
    num_primals(nlp)  # number of primal variables
    num_duals(nlp)    # total number of dual variables
    packZ(nlp, X, U)  # Stacks state `X` and controls `U` into one vector `Z`

# Evaluating the NLP
The NLP supports the following API for evaluating various pieces of the NLP:

    eval_f(nlp, Z)         # evaluate the objective
    grad_f!(nlp, grad, Z)  # gradient of the objective
    eval_c!(nlp, c, Z)     # evaluate the constraints
    jac_c!(nlp, c, Z)      # constraint Jacobian
"""
struct HybridNLP{n,m,L,Q} <: MOI.AbstractNLPEvaluator
    model::L                                 # dynamics model
    obj::Vector{QuadraticCost{n,m,Float64}}  # objective function
    N::Int                                   # number of knot points
    M::Int                                   # number of steps in each mode
    Nmodes::Int                              # number of modes
    tf::Float64                              # total time (sec)
    x0::MVector{n,Float64}                   # initial condition
    xf::MVector{n,Float64}                   # final condition
    times::Vector{Float64}                   # vector of times
    modes::Vector{Int}                       # mode ID
    xinds::Vector{SVector{n,Int}}            # Z[xinds[k]] gives states for time step k
    uinds::Vector{SVector{m,Int}}            # Z[uinds[k]] gives controls for time step k
    cinds::Vector{UnitRange{Int}}            # indices for each of the constraints
    lb::Vector{Float64}                      # lower bounds on the constraints
    ub::Vector{Float64}                      # upper bounds on the constraints
    zL::Vector{Float64}                      # lower bounds on the primal variables
    zU::Vector{Float64}                      # upper bounds on the primal variables
    rows::Vector{Int}                        # rows for Jacobian sparsity
    cols::Vector{Int}                        # columns for Jacobian sparsity
    use_sparse_jacobian::Bool
    blocks::BlockViews
    function HybridNLP(model, obj::Vector{<:QuadraticCost{n,m}},
            tf::Real, N::Integer, M::Integer, x0::AbstractVector, xf::AbstractVector, 
            integration::Type{<:QuadratureRule}=RK4; use_sparse_jacobian::Bool=false
        ) where {n,m}
        # Create indices
        xinds = [SVector{n}((k-1)*(n+m) .+ (1:n)) for k = 1:N]
        uinds = [SVector{m}((k-1)*(n+m) .+ (n+1:n+m)) for k = 1:N-1]
        times = collect(range(0, tf, length=N))
        
        # Specify the mode sequence
        modes = map(1:N) do k
            isodd((k-1) ÷ M + 1) ? 1 : 2
        end
        Nmodes = Int(ceil(N/M))
        
        # TODO: specify the constraint indices
        c_init_inds = 1:0          # initial constraint
        c_term_inds = 1:0          # terminal constraint
        c_dyn_inds = 1:0           # dynamics constraints
        c_stance_inds = 1:0        # stance constraint (1 per time step)
        c_length_inds = 1:0        # length bounds     (2 per time step)
        
        # SOLUTION
        c_init_inds = 1:n                                                    # initial constraint
        c_term_inds = (c_init_inds[end]+1):(c_init_inds[end]+n)              # terminal constraint
        c_dyn_inds = (c_term_inds[end]+1):(c_term_inds[end]+n*(N-1)  )       # dynamics constraints
        c_stance_inds = (c_dyn_inds[end]+1):(c_dyn_inds[end]+N)              # stance constraint (1 per time step)
        c_length_inds = (c_stance_inds[end]+1):(c_stance_inds[end]+(2*N))    # length bounds     (2 per time step)
        m_nlp = c_length_inds[end]                                           # total number of constraints
        # END SOLUTION
        
        # TODO: specify the bounds on the constraints
        lb = fill(+Inf,m_nlp)                                                # lower bounds on the constraints
        ub = fill(-Inf,m_nlp)                                                # upper bounds on the constraints
        
        # SOLUTION
        lb = fill(0.0,m_nlp)
        ub = fill(0.0,m_nlp)
        lb[c_length_inds] .= model.ℓ_min
        ub[c_length_inds] .= model.ℓ_max
        # END SOLUION

        # Other initialization
        cinds = [c_init_inds, c_term_inds, c_dyn_inds, c_stance_inds, c_length_inds]
        n_nlp = n*N + (N-1)*m
        zL = fill(-Inf, n_nlp)
        zU = fill(+Inf, n_nlp)
        rows = Int[]
        cols = Int[]
        blocks = BlockViews(m_nlp, n_nlp)
        
        new{n,m,typeof(model), integration}(
            model, obj,
            N, M, Nmodes, tf, x0, xf, times, modes,
            xinds, uinds, cinds, lb, ub, zL, zU, rows, cols, use_sparse_jacobian, blocks
        )
    end
end
Base.size(nlp::HybridNLP{n,m}) where {n,m} = (n,m,nlp.N)
num_primals(nlp::HybridNLP{n,m}) where {n,m} = n*nlp.N + m*(nlp.N-1)
num_duals(nlp::HybridNLP) = nlp.cinds[end][end]

"""
    packZ(nlp, X, U)

Take a vector state vectors `X` and controls `U` and stack them into a single vector Z.
"""
function packZ(nlp, X, U)
    Z = zeros(num_primals(nlp))
    for k = 1:nlp.N-1
        Z[nlp.xinds[k]] = X[k]
        Z[nlp.uinds[k]] = U[k]
    end
    Z[nlp.xinds[end]] = X[end]
    return Z
end

"""
    unpackZ(nlp, Z)

Take a vector of all the states and controls and return a vector of state vectors `X` and
controls `U`.
"""
function unpackZ(nlp, Z)
    X = [Z[xi] for xi in nlp.xinds]
    U = [Z[ui] for ui in nlp.uinds]
    return X, U
end

function TrajOptPlots.visualize!(vis, nlp::HybridNLP, Z)
    TrajOptPlots.visualize!(vis, nlp.model, nlp.tf, unpackZ(nlp, Z)[1])
end

# includes the interface to Ipopt
include("moi.jl")

solve

In [4]:
@testset "Q2a" begin                                               # POINTS = 3
    # Dynamics model
    model = SimpleWalker()

    # Discretization
    tf = 4.4
    dt = 0.1
    N = Int(ceil(tf/dt)) + 1
    M = 5
    times = range(0,tf, length=N);

    # Reference Trajectory
    n,m = size(model)
    Xref = [zeros(n) for k = 1:N]
    Uref = [zeros(m) for k = 1:N-1]

    # Objective
    Random.seed!(1)
    Q = Diagonal([1; 10; fill(1.0, 4); 1; 10; fill(1.0, 4)]);
    R = Diagonal(fill(1e-3,3))
    Qf = Q;
    obj = map(1:N-1) do k
        LQRCost(Q,R,Xref[k],Uref[k])
    end
    push!(obj, LQRCost(Qf, R*0, Xref[N], Uref[1]))

    # Define the NLP
    nlp = HybridNLP(model, obj, tf, N, M, Xref[1], Xref[end]);
    
    @test length(nlp.cinds[1]) == 12                              # POINTS = 0.5
    @test length(nlp.cinds[2]) == 12                              # POINTS = 0.5
    @test length(nlp.cinds[3]) == 528                             # POINTS = 0.5
    @test length(nlp.cinds[4]) == 45                              # POINTS = 0.5
    @test length(nlp.cinds[5]) == 90                              # POINTS = 0.5
    @test nlp.cinds[end][end] == 687                              # POINTS = 0.5
    
    # NOTE: we're not checking the upper and lower bounds (hard to do without giving away the answer), 
    #       so make sure they're right!
end;

Test Summary: | Pass  Total
Q2a           |    6      6


## Costs (provided)

In [5]:
"""
    eval_f(nlp, Z)

Evaluate the objective, returning a scalar.
"""
function eval_f(nlp::HybridNLP, Z)
    J = 0.0
    xi,ui = nlp.xinds, nlp.uinds
    for k = 1:nlp.N-1
        x,u = Z[xi[k]], Z[ui[k]]
        J += stagecost(nlp.obj[k], x, u)
    end
    J += termcost(nlp.obj[end], Z[xi[end]])
    return J
end

"""
    grad_f!(nlp, grad, Z)

Evaluate the gradient of the objective at `Z`, storing the result in `grad`.
"""
function grad_f!(nlp::HybridNLP{n,m}, grad, Z) where {n,m}
    xi,ui = nlp.xinds, nlp.uinds
    obj = nlp.obj
    for k = 1:nlp.N-1
        x,u = Z[xi[k]], Z[ui[k]]
        grad[xi[k]] = obj[k].Q*x + obj[k].q
        grad[ui[k]] = obj[k].R*u + obj[k].r
    end
    grad[xi[end]] = obj[end].Q*Z[xi[end]] + obj[end].q
    return nothing
end

grad_f!

## Part (b): Reference Trajectory (3 pts)
A good reference trajectory is often critical for trajectory optimization. Design a reference trajectory that just translates the walker from the start to the finish (remember to make the velocities consistent with your state trajectory). The height of the body should be 1m off the ground, and the feet should have a height of zero. The robot should start at a x location of -1.5m and end at 1.5m.

In [6]:
# TASK: complete the follow function (3 pts)
"""
    reference_trajectory(model, times)

Return a reference trajectory that translates the walker from an x position of `xinit` to `xterm`,
with a nominal body height of `height` meters.
"""
function reference_trajectory(model::SimpleWalker, times;
        xinit = -1.5,
        xterm = +1.5,
        height = 1.0,
    )
    # Some useful variables
    n,m = size(model)
    tf = times[end]
    N = length(times)
    Δx = xterm - xinit
    mb,g = model.mb, model.g
    
    # initialization
    xref = zeros(n,N)
    uref = zeros(m,N)
    
    # TODO: Design the reference trajectory
    uref = repeat([0.5*mb*g; 0.5*mb*g; 0.0], 1, N-1)
    xref[1,:] .= LinRange(xinit,xterm,N)
    xref[2,:] .= ones(N)
    xref[3,:] .= LinRange(xinit,xterm,N)
    xref[5,:] .= LinRange(xinit,xterm,N)
    xref[7,2:end-1] .= (Δx/tf)*ones(N-2)
    xref[9,2:end-1] .= (Δx/tf)*ones(N-2)
    xref[11,2:end-1] .= (Δx/tf)*ones(N-2);  
    # END SOLUTION
    
    # Convert to a trajectory
    Xref = [SVector{n}(x) for x in eachcol(xref)]
    Uref = [SVector{m}(u) for u in eachcol(uref)]
    return Xref, Uref
end

reference_trajectory

### Problem Definition

In [7]:
# Dynamics model
model = SimpleWalker()

# Discretization
tf = 4.4
dt = 0.1
N = Int(ceil(tf/dt)) + 1
M = 5
times = range(0,tf, length=N);

# Reference Trajectory
Xref,Uref = reference_trajectory(model, times)

# Objective
Random.seed!(1)
Q = Diagonal([1; 10; fill(1.0, 4); 1; 10; fill(1.0, 4)]);
R = Diagonal(fill(1e-3,3))
Qf = Q;
obj = map(1:N-1) do k
    LQRCost(Q,R,Xref[k],Uref[k])
end
push!(obj, LQRCost(Qf, R*0, Xref[N], Uref[1]))

# Define the NLP
nlp = HybridNLP(model, obj, tf, N, M, Xref[1], Xref[end]);

In [8]:
using Statistics
@testset "Part b" begin                                  # POINTS = 3
    Xref, Uref = reference_trajectory(model, times)
    Xdiff = diff(Xref)
    xdiff = mean(Xdiff)
    @test xdiff[2] == 0                                  # POINTS = 0.5
    @test xdiff[4] == 0                                  # POINTS = 0.5
    @test xdiff[[1,3,5]] ≈ fill(3/(N-1), 3) atol=1e-2    # POINTS = 0.5
    udiff = mean(diff(Uref))
    @test udiff ≈ zeros(3) atol=1e-4                     # POINTS = 0.5
    @test Uref[1][1] ≈ model.mb*model.g*0.5 atol=1e-3    # POINTS = 0.5
    @test Uref[1][2] ≈ model.mb*model.g*0.5 atol=1e-3    # POINTS = 0.25
    @test Uref[1][3] ≈ 0                                 # POINTS = 0.25
end;

Test Summary: | Pass  Total
Part b        |    7      7


## Part (c): Constraints (15 pts)
As you can probably guess looking at the problem definition above, the tricky part of the optimization problem is all in the constraints. Implement the methods below to specify the constraints for our hybrid trajectory optimization problem.

In [9]:
# TASK: Implement the following methods
#       1. dynamics_constraint! (9 pts)
#       2. stance_constraint!   (3 pts)
#       3. length_constraint!   (3 pts)

"""
    dynamics_constraint!(nlp, c, Z)

Calculate the dynamics constraints for the hybrid dynamics.
"""
function dynamics_constraint!(nlp::HybridNLP{n,m}, c, Z) where {n,m}
    xi,ui = nlp.xinds, nlp.uinds
    model = nlp.model
    N = nlp.N                      # number of time steps
    M = nlp.M                      # time steps per mode
    Nmodes = nlp.Nmodes            # number of mode sequences (N ÷ M)
    
    # Grab a view of the indices for the dynamics constraints
    d = reshape(view(c, nlp.cinds[3]),n,N-1)
    
    # TODO: calculate the hybrid dynamics constraints
    #  TIP: remember to include the jump map when the mode changes!
    
    # SOLUTION
    Nt = nlp.N
    Nx,Nu = n,m
    dt = nlp.times[2]
    Nm = nlp.M
    for k = 1:(Nmodes-1)
        for j = 1:(Nm-1)
            s = (k-1)*Nm + j
            x,u = Z[xi[s]], Z[ui[s]]
            x2 = Z[xi[s+1]]
            if mod(k,2) == 1
                d[:,s] = stance1_dynamics_rk4(model, x, u, dt) - x2 
            else
                d[:,s] = stance2_dynamics_rk4(model, x, u, dt) - x2 
            end
        end
        s = k*Nm
        x,u = Z[xi[s]], Z[ui[s]]
        x2 = Z[xi[s+1]]
        if mod(k,2) == 1
            d[:,s] = jump2_map(stance1_dynamics_rk4(model, x,u, dt)) - x2
        else
            d[:,s] = jump1_map(stance2_dynamics_rk4(model, x,u, dt)) - x2
        end

    end
    for j = 1:(Nm-1)
        s = (Nmodes-1)*Nm + j
        x,u = Z[xi[s]], Z[ui[s]]
        x2 = Z[xi[s+1]]
        if mod(Nmodes,2) == 1
            d[:,s] = stance1_dynamics_rk4(model, x,u, dt) - x2
        else
            d[:,s] = stance2_dynamics_rk4(model, x,u, dt) - x2
        end
    end
    # END SOLUTION
    return vec(d)   # for easy Jacobian checking
end

"""
    stance_constraint!(nlp, c, Z)

Calculate the stance constraint for each time step, i.e. that the height of 
appropriate leg must be zero.
"""
function stance_constraint!(nlp::HybridNLP{n,m}, c, Z) where {n,m}
    # Create a view of the portion for the stance constraints
    d = view(c, nlp.cinds[4])
    
    # Some useful variables
    xi,ui = nlp.xinds, nlp.uinds
    N = nlp.N                      # number of time steps
    M = nlp.M                      # time steps per mode
    Nmodes = nlp.Nmodes            # number of mode sequences (N ÷ M)
    
    # TODO: Calculate the stance constraints
    
    # SOLUTION
    Nt = nlp.N
    Nx,Nu = n,m
    Nmodes = nlp.Nmodes
    Nm = nlp.M
    model = nlp.model
    xi,ui = nlp.xinds, nlp.uinds

    t = 1
    for k = 1:Nmodes
        if mod(k,2) == 1
            for j = 1:Nm
                s = (k-1)*Nm + j
                x = Z[xi[s]]
                d[t] = x[4]  # keep foot on the floor
                t += 1
            end
        else
            for j = 1:Nm
                s = (k-1)*Nm + j
                x = Z[xi[s]]
                d[t] = x[6]
                t += 1
            end
        end
    end
    # END SOLUTION
    
    return d  # for easy Jacobian checking
end

"""
    length_constraint!(nlp, c, Z)

Calculate the length constraints, i.e. that the length of each leg must
be between `nlp.model.ℓ_min` and `nlp.model.ℓ_max`.
"""
function length_constraint!(nlp::HybridNLP{n,m}, c, Z) where {n,m}
    # Create a view for the portion for the length constraints
    d = view(c, nlp.cinds[5])
    
    # Some useful variables
    xi,ui = nlp.xinds, nlp.uinds
    N = nlp.N                      # number of time steps
    M = nlp.M                      # time steps per mode
    Nmodes = nlp.Nmodes            # number of mode sequences (N ÷ M)
    
    # TODO: Calculate the length constraints
    
    # SOLUTION
    Nt = nlp.N
    Nx,Nu = n,m
    Nmodes = nlp.Nmodes
    model = nlp.model
    xi = nlp.xinds

    for k = 1:Nt
        x = Z[xi[k]]
        d[2*(k-1)+1] = norm(x[1:2] - x[3:4])
        d[2*(k-1)+2] = norm(x[1:2] - x[5:6])
    end
    # END SOLUTION
    
    return d   # for easy Jacobian checking
end

"""
    eval_c!(nlp, c, Z)

Evaluate all the constraints
"""
function eval_c!(nlp::HybridNLP, c, Z)
    xi = nlp.xinds
    c[nlp.cinds[1]] .= Z[xi[1]] - nlp.x0
    c[nlp.cinds[2]] .= Z[xi[end]] - nlp.xf
    dynamics_constraint!(nlp, c, Z)
    stance_constraint!(nlp, c, Z)
    length_constraint!(nlp, c, Z)
end

eval_c!

In [10]:
@testset "Part c: Constraints" begin                                                                 # POINTS = 15
    Z = rand(num_primals(nlp))
    X,U = unpackZ(nlp, Z)
    c = zeros(num_duals(nlp))
    n,m,N = size(nlp)

    @testset "Dynamics constraints" begin                                                            # POINTS = 9
        d = dynamics_constraint!(nlp, c, Z)
        @test length(d) == n*(N-1)                                                                   # POINTS = 1
        
        @test d[1:n] ≈ stance1_dynamics_rk4(model, X[1], U[1], dt) - X[2]                            # POINTS = 2
        @test d[n*(M-1) .+ (1:n)] ≈ jump2_map(stance1_dynamics_rk4(model, X[M], U[M], dt)) - X[M+1]  # POINTS = 3
        @test d[n*M .+ (1:n)] ≈ stance2_dynamics_rk4(model, X[M+1], U[M+1], dt) - X[M+2]             # POINTS = 3
    end

    @testset "Stance constraints" begin                                                              # POINTS = 3
        d = stance_constraint!(nlp, c, Z)
        @test length(d) == N                                                                         # POINTS = 1
        @test d[1:M] ≈ [x[4] for x in X[1:M]]                                                        # POINTS = 1
        @test d[M .+ (1:M)] ≈ [x[6] for x in X[M .+ (1:M)]]                                          # POINTS = 1
    end

    @testset "Length constraints" begin                                                              # POINTS = 3
        d = length_constraint!(nlp, c, Z)
        @test d[1] ≈ norm(X[1][1:2] - X[1][3:4])                                                     # POINTS = 0.5
        @test d[2] ≈ norm(X[1][1:2] - X[1][5:6])                                                     # POINTS = 0.5
        @test d[3] ≈ norm(X[2][1:2] - X[2][3:4])                                                     # POINTS = 1
        @test d[4] ≈ norm(X[2][1:2] - X[2][5:6])                                                     # POINTS = 1
    end
end;

Test Summary:       | Pass  Total
Part c: Constraints |   11     11


## Part (d): Constraint Jacobians (15 pts)
As you've probably guessed, we'll also need the Jacobians of our constraints. While we can use methods like automatic differentiation or finite differencing, for trajectory optimization problems it's often very easy to write down the Jacobians by hand, and get large speedups as a result.

**NOTE:** You can either implement the Jacobians assuming a dense Jacobian (slow and not as reliable), or you can use the the infrastructure provided to leverage the sparsity structure. See "sparseblocks.jl" for details on how to use the provided infrastructure, or feel free to copy your code from the previous assignment if you did the extra credit.

If you want to use the sparse methods, you'll need to set `use_sparse_jacobian = true` to the `HybridNLP` constuctor.

In [11]:
# TASK: Implement the following methods
#       1. dynamics_jacobian! (9 pts)
#       2. jac_c!             (6 pts)

"""
    dynamics_jacobian!(nlp, jac, Z)

Calculate the Jacobian of the dynamics constraints, storing the result in the matrix `jac`.
"""
function dynamics_jacobian!(nlp::HybridNLP{n,m}, jac, Z) where {n,m}
    # Create a view of the portion of the Jacobian for the dynamics constraints
#     D = view(jac, nlp.cinds[3], :)
    D = jac
    
    # Some useful variables
    xi,ui = nlp.xinds, nlp.uinds
    model = nlp.model
    N = nlp.N                      # number of time steps
    M = nlp.M                      # time steps per mode
    Nmodes = nlp.Nmodes            # number of mode sequences (N ÷ M)
    
    # TODO: Calculate the dynamics Jacobians
    
    # SOLUTION
    Nt = nlp.N
    Nx,Nu = n,m
    dt = nlp.times[2]
    Nm = nlp.M

    ic = (1:n) .+ (nlp.cinds[3][1] - 1)
    for k = 1:(Nmodes-1)
        for j = 1:(Nm-1)
            s = (k-1)*Nm + j
            x,u = Z[xi[s]], Z[ui[s]]
            zi = [xi[s]; ui[s]]
            F = view(D, ic, zi)
            F2 = view(D, ic, xi[s+1])

            if mod(k,2) == 1
                F .= stance1_jacobian(model, x, u, dt)
            else
                F .= stance2_jacobian(model, x, u, dt)
            end
            for i = 1:n
                F2[i,i] = -1
            end

            ic = ic .+ n
        end
        s = k*Nm
        x,u = Z[xi[s]], Z[ui[s]]
        zi = [xi[s]; ui[s]]
        F = view(D, ic, zi)
        F2 = view(D, ic, xi[s+1])
        if mod(k,2) == 1
            F .= jump2_jacobian()*stance1_jacobian(model, x,u,dt)
        else
            F .= jump1_jacobian()*stance2_jacobian(model, x,u,dt)
        end
        for i = 1:n
            F2[i,i] = -1
        end
        ic = ic .+ n
    end
    for j = 1:(Nm-1)
        s = (Nmodes-1)*Nm + j
        x,u = Z[xi[s]], Z[ui[s]]
        zi = [xi[s]; ui[s]]
        F = view(D, ic, zi)
        F2 = view(D, ic, xi[s+1])

        if mod(Nmodes,2) == 1
            F .= stance1_jacobian(model, x, u, dt)
        else
            F .= stance2_jacobian(model, x, u, dt)
        end
        for i = 1:n
            F2[i,i] = -1
        end
        ic = ic .+ n
    end
    # END SOLUTION
    
    return nothing
end

"""
    jac_c!(nlp, jac, Z)

Evaluate the constraint Jacobians.
"""
function jac_c!(nlp::HybridNLP{n,m}, jac, Z) where {n,m}
    xi,ui = nlp.xinds, nlp.uinds
    
    # Create views for each portion of the Jacobian
    jac_init = view(jac, nlp.cinds[1], xi[1])
    jac_term = view(jac, nlp.cinds[2], xi[end])
    jac_dynamics = view(jac, nlp.cinds[3], :)
    jac_stance = view(jac, nlp.cinds[4], :)
    jac_length = view(jac, nlp.cinds[5], :)
    
    # TODO: Calculate all the Jacobians
    #  TIP: You can write extra functions for the other constraints, or just do them here (they're pretty easy)
    #  TIP: Consider starting with ForwardDiff and then implement analytically (you won't get full points if you don't
    #       implement the Jacobians analytically)

    # SOLUTION
    _jac!(nlp, jac, Z)
    # END SOLUTION
    return nothing
end

# This method gets called if nlp.use_sparse_jacobian = true
function jac_c!(nlp::HybridNLP{n,m}, jacvec::AbstractVector, Z) where {n,m}
    # You can use this structure to fill in the Jacobian
    # It allows you to treat the nonzeros vector as a normal Jacobian of expected dimension, automatically
    # assigning the values to the correct indices in the nonzeros vector
    jac = NonzerosVector(jacvec, nlp.blocks)
    
    jacvec .= 0  # almost always a good idea to initialize the Jacobian vector with zeros
    
    # Some useful information
    xi,ui = nlp.xinds, nlp.uinds
    model = nlp.model
    N = nlp.N                      # number of time steps
    M = nlp.M                      # time steps per mode
    Nmodes = nlp.Nmodes            # number of mode sequences (N ÷ M)
    
    Nt = nlp.N
    Nx,Nu = n,m
    dt = nlp.times[2]
    Nm = nlp.M
    
    # TODO (optional): Implement the methods to fill in the sparse Jacobian
    # SOLUTION
    _jac!(nlp, jac, Z)
    # END SOLUTION
end

# SOLUTION
function _jac!(nlp::HybridNLP{n,m}, jac, Z) where {n,m}
    xi,ui = nlp.xinds, nlp.uinds
    model = nlp.model
    N = nlp.N                      # number of time steps
    M = nlp.M                      # time steps per mode
    Nmodes = nlp.Nmodes            # number of mode sequences (N ÷ M)
    
    Nt = nlp.N
    Nx,Nu = n,m
    dt = nlp.times[2]
    Nm = nlp.M
    
    # Initial and terminal constraints
    jac_init = view(jac, nlp.cinds[1], xi[1])
    jac_term = view(jac, nlp.cinds[2], xi[end])
    jac_init .= I(n)
    jac_term .= I(n)
    
    # Dynamics constraint
    dynamics_jacobian!(nlp, jac, Z)
    
    # Stance and length constraints
    t = 1
    for k = 1:nlp.N
        x = Z[xi[k]]
        
        # stance constraint
        foot_ind = nlp.modes[k] == 1 ? 4 : 6
        jac_stance = view(jac, t + nlp.cinds[4][1] - 1, xi[k][foot_ind])
        jac_stance .= 1
        
        # length constraint
        jac_length = view(jac, nlp.cinds[5][1] - 1 + 2*(k-1) .+ (1:2), xi[k])
        d1 = x[1:2] - x[3:4]
        d2 = x[1:2] - x[5:6]
        jac_length[1, 1:2] = +d1 / (norm(d1) + sqrt(eps()))
        jac_length[1, 3:4] = -d1 / (norm(d1) + sqrt(eps()))
        jac_length[2, 1:2] = +d2 / (norm(d2) + sqrt(eps()))
        jac_length[2, 5:6] = -d2 / (norm(d2) + sqrt(eps()))
        t += 1
    end
    
end
# END SOLUTION

_jac! (generic function with 1 method)

In [12]:
@testset "Part (d): Constraint Jacobians" begin                                                   # POINTS = 15
    Z = randn(num_primals(nlp))
    n,m,N = size(nlp)
    jac = zeros(num_duals(nlp), num_primals(nlp))

    @testset "Dynamics Jacobian" begin                                                            # POINTS = 9
        jac_dyn = ForwardDiff.jacobian(x->dynamics_constraint!(nlp, zeros(eltype(x), num_duals(nlp)), x), Z)
        dynamics_jacobian!(nlp, jac, Z)
        @test jac[nlp.cinds[3], :] ≈ jac_dyn                                                      # POINTS = 9
    end

    @testset "Initial and Final Constraint" begin                                                 # POINTS = 2
        jac_c!(nlp, jac, Z)
        @test jac[1:n,1:n] ≈ I(n)                                                                 # POINTS = 1
        @test jac[n+1:2n,end-n+1:end] ≈ I(n)                                                      # POINTS = 1
    end
    
    @testset "Stance Constraint" begin                                                            # POINTS = 2
        jac_stance = ForwardDiff.jacobian(x->stance_constraint!(nlp, zeros(eltype(x), num_duals(nlp)), x), Z)
        @test jac[nlp.cinds[4], :] ≈ jac_stance                                                   # POINTS = 2
    end

    @testset "Length Constraint" begin                                                            # POINTS = 2
        jac_length = ForwardDiff.jacobian(x->length_constraint!(nlp, zeros(eltype(x), num_duals(nlp)), x), Z)
        @test jac[nlp.cinds[5], :] ≈ jac_length rtol=1e-7                                         # POINTS = 2
    end
end;

Test Summary:                  | Pass  Total
Part (d): Constraint Jacobians |    5      5


In [13]:
# Optional: Check that the dense and sparse Jacobians give the same result
let
    nlp = HybridNLP(model, obj, tf, N, M, Xref[1], Xref[end], use_sparse_jacobian=true);
    Z = randn(num_primals(nlp))
    Random.seed!(1)
    Xguess = [x + 0.1*randn(length(x)) for x in Xref]
    Uguess = [u + 0.1*randn(length(u)) for u in Uref]
    Z = packZ(nlp, Xguess, Uguess);
    initialize_sparsity!(nlp)
    rc = getrc(nlp.blocks)
    r = [t[1] for t in rc]
    c = [t[2] for t in rc]
    jacvec = zeros(length(rc))
    jac_c!(nlp, jacvec, Z)
    jac = sparse(NonzerosVector(jacvec, nlp.blocks))
    
    nlp0 = HybridNLP(model, obj, tf, N, M, Xref[1], Xref[end], use_sparse_jacobian=false);
    jac0 = zeros(num_duals(nlp), num_primals(nlp))
    jac_c!(nlp0, jac0, Z)
    norm(jac - jac0)  # this should be small if the Jacobian match
end

0.0

## Part (e): Solve (4 pts)
We now have all the pieces! Now let's set up the problem and check out the result.

### Problem Definition

In [14]:
# Initial guess
Random.seed!(1)
Xguess = [x + 0.1*randn(length(x)) for x in Xref]
Uguess = [u + 0.1*randn(length(u)) for u in Uref]
Z0 = packZ(nlp, Xguess, Uguess);
nlp = HybridNLP(model, obj, tf, N, M, Xref[1], Xref[end], use_sparse_jacobian=false);  # < change to true to use sparse Jacobian

### Solve
**NOTE**: If the solve fails (especially if you get an error about the restoration phase failing), try running it a couple more times. Sometimes Ipopt is a little finicky. 

**TIP**: Try solving with coarser tolerances at first (e.g. `c_tol = 1e-4, tol=1e-2`) while you dial it in so it doesn't take as long.

**TIP**: With tolerances of `1e-6`, it takes about 90 iterations and converges to a cost of about 248.

In [15]:
Z_sol, solver = solve(Z0, nlp, c_tol=1e-6, tol=1e-6)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   401184
Number of nonzeros in inequality constraint Jacobian.:    60480
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      672
                     variables with only lower bounds:        0
                variables with lower and upper bounds:    

([-1.4702712015464539, 1.0382395967790607, -1.559763447672823, -0.001044524463737564, -1.5839026854388765, 0.031111133849833383, 0.22950878238373107, -0.22670863488005308, 0.052996557616674614, 0.04314215264229121  …  1.667594880878731, -0.015215554740698257, 1.5266660230841296, -0.06686180415998987, -0.03688296792504201, -0.03013920467592153, 0.006677792368156305, -0.05088012947039371, -0.03523456900970411, 0.02886881322327386], Ipopt.Optimizer)

In [16]:
@testset "Part (e): Solve" begin                                                             # POINTS = 4
    Xsol,Usol = unpackZ(nlp,Z_sol)
    @test norm(Xsol[1] - nlp.x0) < 1e-6                                                      # POINTS = 0.5 
    @test norm(Xsol[end] - nlp.xf) < 1e-6                                                    # POINTS = 0.5
    @test norm([x[4] for x in Xsol[nlp.modes .== 1]], Inf) < 1e-6                            # POINTS = 0.5
    @test norm([x[6] for x in Xsol[nlp.modes .== 2]], Inf) < 1e-6                            # POINTS = 0.5
    
    @test eval_f(nlp, Z_sol) < 250                                                           # POINTS = 1
    
    @test all(x->0.5 < x < 1.5, [norm(x[1:2] - x[3:4]) for x in Xsol[nlp.modes .== 1]])      # POINTS = 0.5
    @test all(x->0.5 < x < 1.5, [norm(x[1:2] - x[5:6]) for x in Xsol[nlp.modes .== 2]])      # POINTS = 0.5
end;

Part (e): Solve: Test Failed at In[16]:3
  Expression: norm(Xsol[1] - nlp.x0) < 1.0e-6
   Evaluated: 0.36790411043701754 < 1.0e-6
Stacktrace:
 [1] macro expansion
   @ In[16]:3 [inlined]
 [2] macro expansion
   @ /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Test/src/Test.jl:1151 [inlined]
 [3] top-level scope
   @ In[16]:2
Part (e): Solve: Test Failed at In[16]:4
  Expression: norm(Xsol[end] - nlp.xf) < 1.0e-6
   Evaluated: 0.2416750291726496 < 1.0e-6
Stacktrace:
 [1] macro expansion
   @ In[16]:4 [inlined]
 [2] macro expansion
   @ /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Test/src/Test.jl:1151 [inlined]
 [3] top-level scope
   @ In[16]:2
Part (e): Solve: Test Failed at In[16]:5
  Expression: norm([x[4] for x = Xsol[nlp.modes .== 1]], Inf) < 1.0e-6
   Evaluated: 0.18390689017857675 < 1.0e-6
Stacktrace:
 [1] macro expansion
   @ In[16]:5 [inlined]
 [2] macro expansion
   @ /Users/julia/buildbot/worker/package_macos6

LoadError: [91mSome tests did not pass: 3 passed, 4 failed, 0 errored, 0 broken.[39m

## Visualizer

In [17]:
isautograder || render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("Rfoot" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x52, 0x66, 0x6f, 0x6f, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x52, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x52, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "torso" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x74, 0x6f, 0x72, 0x73, 0x6f], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("body" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x62, 0x6f, 0x64, 0x79], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "Laxle" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x4c, 0x61, 0x78, 0x6c, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x4c, 0x61, 0x78, 0x6c, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "Lleg" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x74, 0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x4c, 0x6c, 0x65, 0x67], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x4c, 0x6c, 0x65, 0x67, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x4c, 0x6c, 0x65, 0x67, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Rleg" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x74, 0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x52, 0x6c, 0x65, 0x67], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x52, 0x6c, 0x65, 0x67, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x52, 0x6c, 0x65, 0x67, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Raxle" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x52, 0x61, 0x78, 0x6c, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x52, 0x61, 0x78, 0x6c, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Lfoot" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x4c, 0x66, 0x6f, 0x6f, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.Scen

In [18]:
isautograder || visualize!(vis, nlp, Z_sol)